In [1]:
import numpy as np
import pandas as pd
import math
import sys, os
#import seaborn as sns
#from scipy import stats
#from pathlib import Path
#import matplotlib.pyplot as plt
#from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.datasets import fetch_california_housing
#import math
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import optuna
from optuna.samplers import TPESampler
import lightgbm as lgbm
#from xgboost import XGBRegressor
#import xgboost as xgb
#from catboost import CatBoostRegressor
#from lightgbm.sklearn import LGBMRegressor
#from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [2]:
train_time = 1 * 60 * 60
objective = 'lgbmregressor'

In [3]:
sys.path.append(os.path.realpath('..'))

In [4]:
train_df = pd.read_csv(r"..\data\train.csv", index_col=0)
test_df = pd.read_csv(r"..\data\test.csv", index_col=0)
sample_sub = pd.read_csv(r"..\data\sample_submission.csv", index_col=0)

In [5]:
del train_df['Over18']
del train_df['EmployeeCount']
del train_df['StandardHours']

In [6]:
columns_to_vectorize = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']
for vector_target in columns_to_vectorize:
    print(vector_target)
    vectorizer = CountVectorizer()
    vectorizer.fit_transform(train_df[vector_target])
    train_df[f'{vector_target}_v'] = vectorizer.transform(train_df[vector_target]).toarray().argmax(axis=1)[:,None]
    vectorizer.fit_transform(test_df[vector_target])
    test_df[f'{vector_target}_v'] = vectorizer.transform(test_df[vector_target]).toarray().argmax(axis=1)[:,None]

BusinessTravel
Department
EducationField
Gender
JobRole
MaritalStatus
OverTime


In [7]:
features = ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction','HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 
       'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',
        'BusinessTravel_v', 'Department_v', 'EducationField_v','Gender_v', 'JobRole_v', 'MaritalStatus_v', 'OverTime_v']
target = ['Attrition']

In [8]:
scaler = MinMaxScaler().fit(train_df[features])
X = scaler.transform(train_df[features])
X_test = scaler.transform(test_df[features])

scaler = MinMaxScaler().fit(train_df[target])
Y = scaler.transform(train_df[target])

In [9]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}

def render_model(param_grid):
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    return reg
    
def train_model(reg, X_train, y_train, X_valid, y_valid):
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)

    preds = reg.predict(train_df[features])
    fpr, tpr, _ = roc_curve(train_df[target], preds)
    roc_auc = auc(fpr, tpr)
    return reg, roc_auc

def objective_v2(trial):
    scores = []
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'max_depth': trial.suggest_int('max_depth', 3, 6), 
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 100),
        #'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        #'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        #'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.1, 5),
        #'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        #'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        #'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 100), 
        #'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = render_model(param_grid)
        reg, roc_auc = train_model(reg, X_train, y_train, X_valid, y_valid)

        scores.append(roc_auc)
    mean_scores = np.mean(scores)
    param_grid_history[mean_scores] = param_grid

    return mean_scores


In [10]:
RANDOM_STATE = 12 
FOLDS = 5
param_grid_history = {}


def objective_v3(trial):
    oof = np.zeros(len(X))
    scores = []
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.9),
        'max_depth': trial.suggest_int('max_depth', 3, 6), 
        #'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 2, 100),
        #'num_leaves': trial.suggest_int('num_leaves', 100, 10000),
        #'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1),
        #'lambda_l1': trial.suggest_float('lambda_l1', 0.1, 1),
        #'lambda_l2': trial.suggest_float('lambda_l2', 0.1, 1),
        #'subsample': trial.suggest_float('subsample', 0.1, 1),
        #'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0.1, 5),
        #'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1),
        #'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1),
        #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10), 
        #'min_child_samples': trial.suggest_int('min_child_samples', 10, 1000), 
        'reg_lambda': trial.suggest_int('reg_lambda', 30, 100), 
        #'n_estimators': trial.suggest_int('n_estimators', 10, 100000), 
    } 
        
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(10*"=", f"Fold={fold+1}", 10*"=")
        X_train, X_valid = train_df.iloc[train_idx][features], train_df.iloc[valid_idx][features]
        y_train , y_valid = train_df[target].iloc[train_idx] , train_df[target].iloc[valid_idx] 
        
        reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)

        reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)

        oof[valid_idx] = reg.predict(X_valid)

    roc_auc = roc_auc_score(Y, oof)
    
    param_grid_history[roc_auc] = param_grid

    return roc_auc

In [11]:
study = optuna.create_study(direction='maximize', sampler=TPESampler(), study_name=objective)
study.optimize(objective_v3, timeout=train_time)

[I 2023-01-19 19:55:56,664] A new study created in memory with name: lgbmregressor


========== Fold=1 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[14]	valid_0's auc: 0.852027
========== Fold=2 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[21]	valid_0's auc: 0.821706
========== Fold=3 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[23]	valid_0's auc: 0.835508
========== Fold=4 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[8]	valid_0's auc: 0.822754
========== Fold=5 ==========
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

Did not meet early stopping. Best iteration is:
[16]	valid_0's auc: 0.758814
========== Fold=1 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[15]	valid_0's auc: 0.83353
========== Fold=2 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[7]	valid_0's auc: 0.831672
========== Fold=3 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[16]	valid_0's auc: 0.832881
========== Fold=4 ==========
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

...

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

========== Fold=1 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[31]	valid_0's auc: 0.844172
========== Fold=2 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[63]	valid_0's auc: 0.827872
========== Fold=3 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[33]	valid_0's auc: 0.869576
========== Fold=4 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[82]	valid_0's auc: 0.872119
========== Fold=5 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[56]	valid_0's auc: 0.801356
========== Fold=1 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[78]	valid_0's auc: 0

D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in 

Did not meet early stopping. Best iteration is:
[32]	valid_0's auc: 0.831334
========== Fold=3 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[46]	valid_0's auc: 0.879153
========== Fold=4 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[98]	valid_0's auc: 0.872542
========== Fold=5 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[52]	valid_0's auc: 0.815169
========== Fold=1 ==========
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[32]	valid_0's auc: 0.841892
========== Fold=2 ==========
Training until validation scores don't improve for 100 rounds



KeyboardInterrupt



In [12]:
parm_grid_bytes = pickle.dumps(param_grid_history)
with open(f"rendered_data/{objective}_bytes.hex", "wb") as binary_file:
    binary_file.write(parm_grid_bytes)

In [13]:
def train(param_grid):
    reg = lgbm.LGBMRegressor(**param_grid, metric='AUC', random_state=1)
    
    X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.20)
    
    reg.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], callbacks=[lgbm.early_stopping(100, verbose=True)], verbose=False)
    
    return reg

percent = math.ceil(len(param_grid_history.keys()) * .1)
top = sorted(list(param_grid_history.keys()))[-percent:]

train_preds = []
test_preds = []
for key in tqdm(top):
    model = train(param_grid_history[key])
    train_preds.append(model.predict(train_df[features]))
    test_preds.append(model.predict(test_df[features]))

train_final_preds = np.stack(train_preds).mean(0)
test_final_preds = np.stack(test_preds).mean(0)

  0%|                                                                                           | 0/21 [00:00<?, ?it/s]D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataC

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[68]	valid_0's auc: 0.805899
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[58]	valid_0's auc: 0.838921
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[45]	valid_0's auc: 0.848733
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[53]	valid_0's auc: 0.810252
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.849935
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[63]	valid_0's auc: 0.806334
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
 38%|███████████████████████████████▌                                                   | 8/21 [00:00<00:00, 30.96it/s]D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataC

Did not meet early stopping. Best iteration is:
[78]	valid_0's auc: 0.823797
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[83]	valid_0's auc: 0.848487
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[56]	valid_0's auc: 0.846959
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[95]	valid_0's auc: 0.826852
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[32]	valid_0's auc: 0.837273
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[65]	valid_0's auc: 0.788138
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[62]	valid_0's auc: 0.858001
Training until validation scores don't improve for 100 rounds


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples

Did not meet early stopping. Best iteration is:
[37]	valid_0's auc: 0.862992
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[61]	valid_0's auc: 0.884758
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[26]	valid_0's auc: 0.835017
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[69]	valid_0's auc: 0.839456
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[48]	valid_0's auc: 0.877632
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[2]	valid_0's auc: 0.866473
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[33]	valid_0's auc: 0.863015


D:\source\repos\venv\Python310\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
D:\source\repos\venv\Python310\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')
100%|██████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 31.54it/s]

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[34]	valid_0's auc: 0.836287


In [14]:
train_submission = pd.DataFrame(data={'id': train_df.index, target[0]: train_final_preds})
train_submission.to_csv(fr'rendered_data/{objective}_train_submission.csv', index=False)

test_submission = pd.DataFrame(data={'id': test_df.index, target[0]: test_final_preds})
test_submission.to_csv(fr'rendered_data/{objective}_test_submission.csv', index=False)